In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key



In [2]:
# Read Files 
Dataset=pd.read_csv("DATA/VSRR_-_State_and_National_Provisional_Counts_for_Live_Births__Deaths__and_Infant_Deaths.csv")
PopData=pd.read_csv("DATA/nst-est2019-01.csv", header=3)


In [3]:
Dataset.head()

,State,Year,Month,Period,Indicator,Data Value
0,UNITED STATES,2018,January,Monthly,Number of Live Births,315000
1,UNITED STATES,2018,February,Monthly,Number of Live Births,284000
2,UNITED STATES,2018,March,Monthly,Number of Live Births,316000
3,UNITED STATES,2018,April,Monthly,Number of Live Births,298000
4,UNITED STATES,2018,May,Monthly,Number of Live Births,321000


In [4]:
#formatting the data to start as certain row and rename location header
PopData.drop(PopData.index[[0,1]])

PopData = PopData.rename(columns={"Unnamed: 0":"State"})

PopData.head()

,State,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,"308,745,538","308,758,105","309,321,666","311,556,874","313,830,990","315,993,715","318,301,008","320,635,163","322,941,311","324,985,539","326,687,501","328,239,523"
1,Northeast,"55,317,240","55,318,443","55,380,134","55,604,223","55,775,216","55,901,806","56,006,011","56,034,684","56,042,330","56,059,240","56,046,620","55,982,803"
2,Midwest,"66,927,001","66,929,725","66,974,416","67,157,800","67,336,743","67,560,379","67,745,167","67,860,583","67,987,540","68,126,781","68,236,628","68,329,004"
3,South,"114,555,744","114,563,030","114,866,680","116,006,522","117,241,208","118,364,400","119,624,037","120,997,341","122,351,760","123,542,189","124,569,433","125,580,448"
4,West,"71,945,553","71,946,907","72,100,436","72,788,329","73,477,823","74,167,130","74,925,793","75,742,555","76,559,681","77,257,329","77,834,820","78,347,268"


In [5]:
#Droping Indicator not equal to "Number of Live Births"

Data_filter = Dataset[Dataset['Indicator'] == "Number of Live Births"]
Data_filter2 = Data_filter[Data_filter['Year'] == 2018]
Data_filter3 = Data_filter2[Data_filter2['Period'] == 'Monthly']

#Validating
#Data_filter3["Year"].unique()
#Data_filter3["Indicator"].unique()
#Data_filter3["Period"].unique()

Data_filter3

,State,Year,Month,Period,Indicator,Data Value
0,UNITED STATES,2018,January,Monthly,Number of Live Births,315000
1,UNITED STATES,2018,February,Monthly,Number of Live Births,284000
2,UNITED STATES,2018,March,Monthly,Number of Live Births,316000
3,UNITED STATES,2018,April,Monthly,Number of Live Births,298000
4,UNITED STATES,2018,May,Monthly,Number of Live Births,321000
...,...,...,...,...,...,...
1489,WASHINGTON,2018,December,Monthly,Number of Live Births,6750
1490,WEST VIRGINIA,2018,December,Monthly,Number of Live Births,1498
1491,WISCONSIN,2018,December,Monthly,Number of Live Births,5045
1492,WYOMING,2018,December,Monthly,Number of Live Births,545


In [6]:
#Groupby state and sum values to get a full year's live births

Grpby_State= Data_filter3.groupby(["State"])
#Grpby_State.get_group('ALABAMA')
Grpby_State.head()

,State,Year,Month,Period,Indicator,Data Value
0,UNITED STATES,2018,January,Monthly,Number of Live Births,315000
1,UNITED STATES,2018,February,Monthly,Number of Live Births,284000
2,UNITED STATES,2018,March,Monthly,Number of Live Births,316000
3,UNITED STATES,2018,April,Monthly,Number of Live Births,298000
4,UNITED STATES,2018,May,Monthly,Number of Live Births,321000
...,...,...,...,...,...,...
1125,WASHINGTON,2018,May,Monthly,Number of Live Births,7374
1126,WEST VIRGINIA,2018,May,Monthly,Number of Live Births,1644
1127,WISCONSIN,2018,May,Monthly,Number of Live Births,5527
1128,WYOMING,2018,May,Monthly,Number of Live Births,622


In [7]:
FY_Births_State= Grpby_State["Data Value"].sum()
FY_Births_State.sort_values(ascending=True)


State
VERMONT                    5467
WYOMING                    6581
DISTRICT OF COLUMBIA       8985
ALASKA                    10064
RHODE ISLAND              10487
DELAWARE                  10578
NORTH DAKOTA              10633
MONTANA                   11523
SOUTH DAKOTA              11919
NEW HAMPSHIRE             11977
MAINE                     12249
HAWAII                    17006
WEST VIRGINIA             18326
PUERTO RICO               21427
IDAHO                     21431
NEW MEXICO                23039
NEBRASKA                  25454
CONNECTICUT               34719
NEVADA                    35683
KANSAS                    36371
MISSISSIPPI               36941
ARKANSAS                  37122
IOWA                      37835
OREGON                    42228
UTAH                      47200
OKLAHOMA                  49681
KENTUCKY                  53919
SOUTH CAROLINA            56559
ALABAMA                   57898
LOUISIANA                 59657
COLORADO                  62905
WI

In [8]:
NewPop2017 = PopData.loc[:,['State', '2017']]
NewPop2017_df = pd.DataFrame(NewPop2017)
Pop2017=NewPop2017_df.dropna(how="any")

Pop2017["State"]=Pop2017["State"].str.upper()
Pop2017.head()

/Users/Akire/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,State,2017
0,UNITED STATES,"324,985,539"
1,NORTHEAST,"56,059,240"
2,MIDWEST,"68,126,781"
3,SOUTH,"123,542,189"
4,WEST,"77,257,329"


In [9]:
merge_data = pd.merge(FY_Births_State, Pop2017, on="State")
merge_data

,State,Data Value,2017
0,ALABAMA,57898,"4,874,486"
1,ALASKA,10064,"739,700"
2,ARIZONA,80802,"7,044,008"
3,ARKANSAS,37122,"3,001,345"
4,CALIFORNIA,454965,"39,358,497"
5,COLORADO,62905,"5,611,885"
6,CONNECTICUT,34719,"3,573,297"
7,DELAWARE,10578,"956,823"
8,DISTRICT OF COLUMBIA,8985,"694,906"
9,FLORIDA,221627,"20,963,613"


In [10]:
#Replace comma in 2017 data

merge_data["2017"]=merge_data["2017"].str.replace(",","")
merge_data.head()

,State,Data Value,2017
0,ALABAMA,57898,4874486
1,ALASKA,10064,739700
2,ARIZONA,80802,7044008
3,ARKANSAS,37122,3001345
4,CALIFORNIA,454965,39358497


In [16]:
#extract digits from string
merge_data["2017"]=merge_data["2017"].str.extract('(^\d*)')
#merge_data["2017"]=merge_data["2017"].str.replace(np.nan,0)
#merge_data[merge_data["2017"]==np.nan]
merge_data["2017"]=merge_data["2017"].astype(int)

In [23]:
#The formula for crude birth rate is: CBR = (b ÷ p) X 1,000.
#crude birth rate (CBR),  number of live births (b), total population (p)
birth=merge_data["Data Value"]
population=merge_data["2017"]

birthrate=(birth/population)*1000
birthrate.map("{:.2f}".format)

0     11.88
1     13.61
2     11.47
3     12.37
4     11.56
5     11.21
6      9.72
7     11.06
8     12.93
9     10.57
10    12.12
11    11.94
12    12.48
13    11.32
14    12.22
15    12.04
16    12.50
17    12.11
18    12.77
19     9.18
20    11.85
21    10.08
22    11.03
23    12.10
24    12.36
25    11.98
26    10.95
27    13.29
28    12.01
29     8.88
30    11.38
31    11.01
32    11.56
33    11.58
34    14.08
35    11.59
36    12.64
37    10.19
38    10.62
39     6.44
40     9.93
41    11.26
42    13.65
43    12.04
44    13.39
45    11.67
46    15.22
47     8.76
48    11.82
49    11.59
50    10.09
51    11.09
52    11.37
dtype: object

In [24]:
merge_data["Birthrate/1000"]=birthrate
merge_data

,State,Data Value,2017,Birthrate/1000
0,ALABAMA,57898,4874486,11.877765
1,ALASKA,10064,739700,13.605516
2,ARIZONA,80802,7044008,11.471026
3,ARKANSAS,37122,3001345,12.368455
4,CALIFORNIA,454965,39358497,11.559512
5,COLORADO,62905,5611885,11.209246
6,CONNECTICUT,34719,3573297,9.716237
7,DELAWARE,10578,956823,11.055336
8,DISTRICT OF COLUMBIA,8985,694906,12.929806
9,FLORIDA,221627,20963613,10.571985
